In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

In [30]:
# ------------------------------------------------------------
# 1. Загрузка данных
# ------------------------------------------------------------

In [31]:
df = pd.read_csv('D:/Users/Николай/Downloads/Х5_with_region_index_2024_only_with_schools_for_test.csv', sep = ';', decimal = ',')

In [32]:
# ------------------------------------------------------------
# 2. Фильтрация несезонного времени
# ------------------------------------------------------------

In [33]:
df = df[df["is_season"] == 0]
df.head()

,store_id,Месяц,traffic_visits,avg_check,"Дата открытия, категориальный",area_group,Населенный пункт,Регион,Численность населения,Количество домохозяйств,traffic,"Трафик авто, в час","Маркетплейсы, доставки, постаматы (100 м)",Медицинские уч. и аптеки (300 м),Школы (300 м),Остановки (300 м),Продуктовые магазины (500 м),Пятерочки (500 м),region_index,traffic_flag,is_season,school_flag
0,0,10,59662,976.170936,Средний по возрасту,Средний,Абинск г,Краснодарский край,38231,728,68.25,146.4,0,0,0,0,0,0,76.58,1,0,0
1,0,5,56674,1025.462154,Средний по возрасту,Средний,Абинск г,Краснодарский край,38231,728,68.25,146.4,0,0,0,0,0,0,76.58,1,0,0
2,0,1,51488,1158.150890,Средний по возрасту,Средний,Абинск г,Краснодарский край,38231,728,68.25,146.4,0,0,0,0,0,0,76.58,1,0,0
3,0,12,59476,1031.000127,Средний по возрасту,Средний,Абинск г,Краснодарский край,38231,728,68.25,146.4,0,0,0,0,0,0,76.58,1,0,0
4,0,9,55856,1035.065767,Средний по возрасту,Средний,Абинск г,Краснодарский край,38231,728,68.25,146.4,0,0,0,0,0,0,76.58,1,0,0


In [34]:
# ------------------------------------------------------------
# 3. Категории регионального индекса
# ------------------------------------------------------------

In [35]:
# ------------------------------------------------------------
# 4. Агрегация по магазинам
# ------------------------------------------------------------

In [36]:
agg_df = (
    df.groupby("store_id")
      .agg({
          "traffic": "mean",
          "avg_check": "mean",
          "school_flag": "max",
          "area_group": "first",
      })
      .reset_index()
)

In [37]:
# ------------------------------------------------------------
# 5. Функция проверки устойчивости + визуализации
# ------------------------------------------------------------

In [38]:
def check_subgroup_stability(data, y_var, area):
    """Проводит t-test, bootstrap, визуализацию и возвращает результаты."""

    group_with = data[data["school_flag"] == 1][y_var]
    group_without = data[data["school_flag"] == 0][y_var]

    # --- t-test ---
    t_stat, p_value = ttest_ind(group_with, group_without, equal_var=False)

    # --- Bootstrap ---
    n_boot = 4000
    boot_diffs = []
    for _ in range(n_boot):
        s1 = group_with.sample(frac=1, replace=True).mean()
        s2 = group_without.sample(frac=1, replace=True).mean()
        boot_diffs.append(s1 - s2)

    ci_low, ci_high = np.percentile(boot_diffs, [2.5, 97.5])
    diff_means = group_with.mean() - group_without.mean()

    # ------------------------------------------------------------
    #  Visualizations
    # ------------------------------------------------------------
    # ========== НАСТРОЙКА ЦВЕТОВ И ШРИФТОВ ==========

    pyaterochka_green = '#008C44'  # Насыщенный зеленый
    light_green = '#E6F4EA'        # Светлый фон
    accent_red = '#E21E26'         # Акцент
    dark_text = '#333333'
    
    # Настройка шрифтов
    plt.rcParams.update({
        'font.family': 'sans-serif',
        'font.sans-serif': ['Arial', 'DejaVu Sans', 'Liberation Sans'],
        'font.size': 11,
        'axes.titlesize': 14,
        'axes.labelsize': 12,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'legend.fontsize': 10,
        'figure.titlesize': 16,
        'text.color': dark_text,
        'axes.labelcolor': dark_text,
        'axes.edgecolor': dark_text,
        'axes.titlecolor': dark_text,
        'xtick.color': dark_text,
        'ytick.color': dark_text
    })
    
    # Используем белый фон для фигур
    plt.rcParams['figure.facecolor'] = 'white'
    plt.rcParams['axes.facecolor'] = 'white'
    
    # ========== Boxplot: распределение Y по группам ==========
    plt.figure(figsize=(7, 5), facecolor='white')
    
    # Создаем boxplot с кастомизацией
    boxplot = plt.boxplot([group_with, group_without], 
                          tick_labels=["School=1", "School=0"],
                          patch_artist=True,  # Разрешаем заливку цветом
                          widths=0.6,
                          showmeans=True,     # Показывать средние значения
                          meanprops={'marker': 'o', 'markerfacecolor': 'none',  # Прозрачная внутренность
                                      'markeredgecolor': light_green,  # Зеленый контур
                                      'markersize': 6, 'alpha': 0.8},
                          medianprops={'color': 'white', 'linewidth': 2.5},  # Белая медиана
                          boxprops={'linewidth': 1.5, 'edgecolor': dark_text},
                          whiskerprops={'color': dark_text, 'linewidth': 1.5},
                          capprops={'color': dark_text, 'linewidth': 1.5},
                          flierprops={'marker': 'o', 'markerfacecolor': 'none',  # Прозрачная внутренность
                                      'markeredgecolor': pyaterochka_green,  # Зеленый контур
                                      'markersize': 6, 'alpha': 0.8})  # Прозрачность только для контура
    
    # Заливаем ящики зеленым цветом БЕЗ прозрачности
    for box in boxplot['boxes']:
        box.set_facecolor(pyaterochka_green)
        box.set_alpha(1.0)  # Без прозрачности

    # Настраиваем заголовок и подписи
    plt.title(f"{y_var}: {area} — boxplot", 
              fontweight='bold', pad=15, color=dark_text)
    plt.ylabel(y_var, fontweight='medium', color=dark_text)
    plt.xlabel("Group", fontweight='medium', color=dark_text)
    
    # Настройка осей
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color(dark_text)
    ax.spines['bottom'].set_color(dark_text)
    
    # Добавляем легкую сетку
    ax.grid(True, axis='y', alpha=0.2, linestyle='--', color=dark_text)
    
    # Улучшаем отступы
    plt.tight_layout()
    
    # Сохраняем с высоким качеством
    plt.savefig(f"plot_boxplot_{y_var}_{area}.png", 
                dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()

    # ========== Bootstrap distribution ==========
    plt.figure(figsize=(7, 5), facecolor='white')
    
    # Создаем гистограмму с кастомизацией БЕЗ прозрачности
    n, bins, patches = plt.hist(boot_diffs, bins=40, 
                                color=pyaterochka_green, 
                                alpha=1.0,  # Без прозрачности
                                edgecolor=dark_text,  # Темный контур
                                linewidth=1.0,
                                density=False)  # Абсолютные частоты
    
    # Линии доверительного интервала с акцентным красным цветом
    plt.axvline(ci_low, linestyle="--", linewidth=2.5, 
                color=accent_red, label=f'Lower CI: {ci_low:.3f}')
    plt.axvline(ci_high, linestyle="--", linewidth=2.5, 
                color=accent_red, label=f'Upper CI: {ci_high:.3f}')
    
    # Линия среднего значения
    mean_diff = np.mean(boot_diffs)
    plt.axvline(mean_diff, linestyle="-", linewidth=2, 
                color=dark_text, alpha=0.8, label=f'Mean: {mean_diff:.3f}')
    
    # Заливаем область доверительного интервала светлым зеленым
    plt.axvspan(ci_low, ci_high, alpha=0.15, color=light_green, label='95% CI')
    
    # Настройка заголовка и подписей
    plt.title(f"Bootstrap difference ({y_var}) — {area}", 
              fontweight='bold', pad=15, color=dark_text)
    plt.xlabel("Difference in means (School=1 - School=0)", 
               fontweight='medium', color=dark_text)
    plt.ylabel("Frequency", fontweight='medium', color=dark_text)
    
    # Настройка осей
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color(dark_text)
    ax.spines['bottom'].set_color(dark_text)
    
    # Добавляем сетку
    ax.grid(True, alpha=0.2, linestyle='--', color=dark_text)
    
    # Добавляем легенду
    legend = plt.legend(loc='best', frameon=True, fancybox=True, 
                       shadow=True, framealpha=0.95, 
                       edgecolor=dark_text)
    legend.get_frame().set_facecolor('white')

    # Добавляем информационный текст
    ci_width = ci_high - ci_low
    info_text = f"CI width: {ci_width:.3f}\nSamples: {len(boot_diffs)}"
    plt.text(0.02, 0.98, info_text, transform=ax.transAxes,
             verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='white', 
                      alpha=0.9, edgecolor=dark_text, pad=0.5),
             fontsize=9)
    
    # Улучшаем отступы
    plt.tight_layout()
    
    # Сохраняем с высоким качеством
    plt.savefig(f"plot_bootstrap_{y_var}_{area}.png", 
                dpi=300, bbox_inches='tight', facecolor='white')
    plt.close()

    return diff_means, p_value, ci_low, ci_high

In [25]:
# ------------------------------------------------------------
# 6. Цикл по всем подгруппам
# ------------------------------------------------------------

In [26]:
results = []

for area in agg_df["area_group"].unique():

    subset = agg_df[(agg_df["area_group"] == area)]

    for y_var in ["traffic", "avg_check"]:

        if subset["school_flag"].nunique() < 2:
            results.append({
                "Area": area,
                "Variable": y_var,
                "Difference": np.nan,
                "p_value": np.nan,
                "CI_low": np.nan,
                "CI_high": np.nan,
                "Note": "Недостаточно вариации flag"
            })
            continue

        diff, p, ci_l, ci_h = check_subgroup_stability(subset, y_var, area)
        results.append({
                        "Area": area,
                        "Variable": y_var,
                        "Difference": diff,
                        "p_value": p,
                        "CI_low": ci_l,
                        "CI_high": ci_h,
                        "Note": ""
                        })

In [27]:
# ------------------------------------------------------------
# 7. Таблица результатов + сохранение
# ------------------------------------------------------------

In [28]:
results_df = pd.DataFrame(results)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
results_df['Note'] = 1*(results_df['p_value'] < 0.05)
results_df.rename(columns={'Note': 'is_significant'}, inplace=True)
results_df.to_csv("stability_results_with_no_region_mimi.csv", index=False)
results_df

,Area,Variable,Difference,p_value,CI_low,CI_high,is_significant
0,Средний,traffic,16.550541,8.546071e-30,13.596467,19.282828,1
1,Средний,avg_check,26.285945,1.278142e-07,16.279380,35.920802,1
2,Маленький,traffic,18.864497,7.728993e-13,13.698348,23.944728,1
3,Маленький,avg_check,-2.659500,6.985314e-01,-16.057142,10.919824,0
4,Большой,traffic,8.671284,2.371222e-03,3.088783,14.330363,1
5,Большой,avg_check,22.367159,4.602881e-02,0.662687,44.512506,1
6,Очень большой,traffic,7.558572,4.770129e-01,-13.031992,28.491219,0
7,Очень большой,avg_check,175.084749,1.577473e-06,104.223657,243.528716,1
